In [2]:
import os
import shutil
import random
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
from PIL import Image
import torch.nn.functional as F
from sklearn.metrics import classification_report


In [3]:
# === PARAMETERS ===
root_folder = 'aug_processed_data'  # Folder with 'healthy' and 'unhealthy' subfolders
base_dir = 'split_data_Resnet'                   # New folder where train/val split will go
train_ratio = 0.8                         # 80% train, 20% val split

In [4]:
def create_train_val_split(root_folder, base_dir, train_ratio=0.8):
    if os.path.exists(base_dir):
        shutil.rmtree(base_dir)  # Clean previous split if exists
    os.makedirs(base_dir)

    train_dir = os.path.join(base_dir, 'train')
    val_dir = os.path.join(base_dir, 'val')

    os.makedirs(train_dir)
    os.makedirs(val_dir)

    classes = [d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]

    for cls in classes:
        os.makedirs(os.path.join(train_dir, cls))
        os.makedirs(os.path.join(val_dir, cls))

        images = os.listdir(os.path.join(root_folder, cls))
        random.shuffle(images)

        train_count = int(len(images) * train_ratio)
        train_imgs = images[:train_count]
        val_imgs = images[train_count:]

        for img_name in train_imgs:
            src = os.path.join(root_folder, cls, img_name)
            dst = os.path.join(train_dir, cls, img_name)
            shutil.copy(src, dst)

        for img_name in val_imgs:
            src = os.path.join(root_folder, cls, img_name)
            dst = os.path.join(val_dir, cls, img_name)
            shutil.copy(src, dst)

    print(f"✅ Dataset split done! Train and val folders created at '{base_dir}'")

In [5]:
# === Run the split function ===
create_train_val_split(root_folder, base_dir, train_ratio)

✅ Dataset split done! Train and val folders created at 'split_data_Resnet'


In [6]:
# === 2. Setup Transfer Learning with the split folders ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder('aug_processed_data', transform=transform)
train_dataset = datasets.ImageFolder(os.path.join(base_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(base_dir, 'val'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

model_resnet = models.resnet18(pretrained=True)
for param in model_resnet.parameters():
    param.requires_grad = False  # Freeze backbone

num_features = model_resnet.fc.in_features
model_resnet.fc = nn.Linear(num_features, len(train_dataset.classes))  # Number of classes

model_resnet = model_resnet.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_resnet.fc.parameters(), lr=0.001)

/usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
# === Training loop ===
def train_model(epochs=10):
    best_val_acc = 0.0
    for epoch in range(epochs):
        model_resnet.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model_resnet(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = 100 * correct / total
        avg_loss = total_loss / len(train_loader)

        model_resnet.eval()
        val_correct, val_total, val_loss = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model_resnet(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_acc = 100 * val_correct / val_total
        avg_val_loss = val_loss / len(val_loader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc

        print(f"Epoch {epoch+1} | Train Loss: {avg_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    print(f"\n🏆 Best Val Accuracy: {best_val_acc:.2f}%")

In [9]:
# === Run training ===
train_model(epochs=10)

Epoch 1 | Train Loss: 0.1999 | Train Acc: 95.62% | Val Loss: 0.4642 | Val Acc: 80.00%
Epoch 2 | Train Loss: 0.1795 | Train Acc: 95.62% | Val Loss: 0.4903 | Val Acc: 82.50%
Epoch 3 | Train Loss: 0.1915 | Train Acc: 95.62% | Val Loss: 0.5859 | Val Acc: 75.00%
Epoch 4 | Train Loss: 0.1587 | Train Acc: 96.25% | Val Loss: 0.4872 | Val Acc: 82.50%
Epoch 5 | Train Loss: 0.1628 | Train Acc: 98.12% | Val Loss: 0.4609 | Val Acc: 80.00%
Epoch 6 | Train Loss: 0.1716 | Train Acc: 95.00% | Val Loss: 0.4951 | Val Acc: 82.50%
Epoch 7 | Train Loss: 0.1619 | Train Acc: 93.75% | Val Loss: 0.5992 | Val Acc: 75.00%
Epoch 8 | Train Loss: 0.1466 | Train Acc: 98.75% | Val Loss: 0.5216 | Val Acc: 80.00%
Epoch 9 | Train Loss: 0.1368 | Train Acc: 98.75% | Val Loss: 0.4567 | Val Acc: 80.00%
Epoch 10 | Train Loss: 0.1286 | Train Acc: 97.50% | Val Loss: 0.4796 | Val Acc: 82.50%

🏆 Best Val Accuracy: 82.50%


Model Testing

In [10]:
def predict_single_image(image_path, model, class_names):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])

    img = Image.open(image_path).convert("RGB")
    img_tensor = transform(img).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(img_tensor)
        probs = F.softmax(output, dim=1)
        _, predicted = torch.max(probs, 1)

    print(f"Predicted Class: {class_names[predicted.item()]}")
    print(f"Class Probabilities: {probs.squeeze().numpy()}")

In [11]:
# Assuming dataset = ImageFolder(...)
class_names = dataset.classes  # ['healthy', 'infected']

# Path to one test image
test_image_path_1 = "processed_data/serie infected leaves/infected_05.png"

predict_single_image(test_image_path_1, model_resnet, class_names)

Predicted Class: series_infected_leaves_augmented
Class Probabilities: [0.40440977 0.59559023]


In [12]:
test_image_path_2 = "processed_data/serie healthy leaves/healthy_05.png"
predict_single_image(test_image_path_2, model_resnet, class_names)

Predicted Class: serie_healthy_leaves_augmented
Class Probabilities: [0.61136645 0.38863352]


Model Evaluation

In [13]:
def evaluate_final_model():
    model_resnet.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model_resnet(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("\n📊 Final Evaluation on Validation Set:")
    print(classification_report(all_labels, all_preds, target_names=val_dataset.classes, digits=2))

# Run this after training
evaluate_final_model()


📊 Final Evaluation on Validation Set:
                                  precision    recall  f1-score   support

  serie_healthy_leaves_augmented       0.88      0.75      0.81        20
series_infected_leaves_augmented       0.78      0.90      0.84        20

                        accuracy                           0.82        40
                       macro avg       0.83      0.82      0.82        40
                    weighted avg       0.83      0.82      0.82        40

